In [1]:
import os

# Add the directory containing the executable to the PATH
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

# Check if the directory is added to the PATH
print(os.environ["PATH"])

/opt/tljh/user/bin:/bin:/usr/bin:/usr/local/cuda/bin


In [2]:
%%bash
nvcc --version
nvprof --version
nsys --version
ncu --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Wed_Apr__9_19:24:57_PDT_2025
Cuda compilation tools, release 12.9, V12.9.41
Build cuda_12.9.r12.9/compiler.35813241_0
nvprof: NVIDIA (R) Cuda command line profiler
Copyright (c) 2012 - 2025 NVIDIA Corporation
Release version 12.9.19 (21)
NVIDIA Nsight Systems version 2025.1.3.140-251335620677v0
NVIDIA (R) Nsight Compute Command Line Profiler
Copyright (c) 2018-2025 NVIDIA Corporation
Version 2025.2.0.0 (build 35613519) (public-release)


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Nov 21 06:42:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 575.51.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           Off |   00000000:00:10.0 Off |                    0 |
| N/A   26C    P0             22W /  250W |       0MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
%%writefile CUDA_cpso.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include <assert.h>
#include <stdint.h>

#define POSITION_MIN -5
#define POSITION_MAX 5
#define VELOCITY_MAX 0.2 * POSITION_MAX
#define VELOCITY_MIN -VELOCITY_MAX // Similar to the experiment setup in the study for rosenbrack

//==============================================================
//                     STRUCT DEFINITIONS
//==============================================================
typedef struct
{
  double position;
  double velocity;
  double personalBest; // Position in dimension
  double personalBestFitness;
} Particle;

typedef struct
{
  Particle *particles;
  double globalBest;
  double globalBestFitness;
} Swarm;

typedef double (*ObjectiveFunction)(size_t, double *);

//==============================================================

//==============================================================
//                     UTILITY FUNCTIONS
//==============================================================
// CPU-function RNG_UNIFORM
static inline double RNG_UNIFORM()
{
  return (double)rand() / (double)RAND_MAX;
}
// CUDA-function RNG
__device__
static inline uint32_t mix32(uint32_t z) {
    z = (z ^ 61u) ^ (z >> 16);
    z *= 9u;
    z = z ^ (z >> 4);
    z *= 0x27d4eb2du;
    z = z ^ (z >> 15);
    return z;
}
__device__
static inline double CUDA_RNG_UNIFORM()
{
  unsigned long long t = clock64();
  uint32_t tid = blockIdx.x * blockDim.x + threadIdx.x;
  uint32_t state = mix32((uint32_t)t ^ tid);

  const uint32_t A = 1664525u;
  const uint32_t C = 1013904223u;
  uint32_t x = state;
  x = A * x + C;        // wraps mod 2^32 automatically
  state = x;

  // Use the top 24 bits for a well-distributed uniform in [0,1)
  double out = ((x >> 8) & 0x00FFFFFF) * (1.0 / 16777216.0);
  return out;
}

__device__ __host__
static inline double square(double b)
{
  return b * b;
}

__device__
static inline double clamp(double value, double min, double max)
{
  if (value < min)
  {
    return min;
  }
  if (value > max)
  {
    return max;
  }
  return value;
}

static inline double randomInRange(double min, double max)
{
  return RNG_UNIFORM() * (max - min) + min;
}

//==============================================================

//==============================================================
//              OBJECTIVE FUNCTIONS
//==============================================================
/**
 * Creates an n-dimensional search space using the Rosenbrock function
 * @param n dimensions
 * @param x coordinates
 */
__device__ __host__
double rosenbrock(size_t n, double *x)
{
  double value = 0.0;

  // f(x) = ∑(i=1 to N−1) [100(x_{i+1} − x_i^2)^2 + (1 − x_i)^2]   where   x = (x_1, …, x_N) ∈ ℝ^N
  for (size_t i = 0; i < n - 1; ++i)
  { 
    value += 100.0 * square(x[i + 1] - square(x[i])) + square(1.0 - x[i]);
  }

  return value;
}
__device__ __host__
double sphere(size_t n, double *x)
{
  double value = 0.0f;
  for (size_t i = 0; i < n; i++)
  {
    value += square(x[i]);
  }

  return value;
}

__device__ __host__
double ackley(size_t n, double *x)
{
  double a = 20.0;
  double b = 0.2;
  double c = 2.0 * M_PI;

  double sumSq = 0.0;
  double sumCos = 0.0;

  for (int i = 0; i < n; i++)
  {
    sumSq += square(x[i]);
    sumCos += cos(c * x[i]);
  }

  double term1 = -a * exp(-b * sqrt(sumSq / n));
  double term2 = -exp(sumCos / n);

  return term1 + term2 + a + exp(1);
}

// Device function-pointer symbols for objective functions
__device__ ObjectiveFunction d_rosenbrock_ptr = rosenbrock;
__device__ ObjectiveFunction d_sphere_ptr     = sphere;
__device__ ObjectiveFunction d_ackley_ptr     = ackley;

//==============================================================

//==============================================================

//==============================================================
//              ALGORITHM-DEFINED FUNCTIONS
//==============================================================
__device__
void b(double *context, size_t n, int j, double z, double *result)
{
  for (int i = 0; i < n; i++)
  {
    if (i == j)
    {
      result[i] = z;
    }
    else
    {
      result[i] = context[i];
    }
  }
}

/**
 * Velocity update function
 * v_{i,j}(t+1) = w v_{i,j}(t) + c_1 r_{1,i}(t) [ y_{i,j}(t) - x_{i,j}(t) ] + c_2 r_{2,i}(t) [ \hat{y}_j(t) - x_{i,j}(t) ]
 *
 */
__device__
void updateVelocity(Particle *particle, double globalBest, int currentIter, int maxIter)
{
  double w = 1.0 - ((double)currentIter / maxIter); // intertia weight, linear scaling
  // printf("w: %f\n", w);
  double c1 = 2; // acceleration coefficient 1
  double c2 = 2; // acceleration coefficient 2
  double r1 = CUDA_RNG_UNIFORM();
  double r2 = CUDA_RNG_UNIFORM();
  double updatedVelocity = (w * particle->velocity) + (c1 * r1 * (particle->personalBest - particle->position)) + (c2 * r2 * (globalBest - particle->position));

  particle->velocity = clamp(updatedVelocity, VELOCITY_MIN, VELOCITY_MAX);
}

/**
 * Position update function
 *
 */
__device__
void updatePosition(Particle *particle)
{
  particle->position += particle->velocity;
  particle->position = clamp(particle->position, POSITION_MIN, POSITION_MAX);
}

__global__
void particleAction(Swarm *swarms, size_t n, int s, double *result, int iter, int maxIter, ObjectiveFunction objectiveFunc)
{

  // particle index per swarm
  size_t tid = threadIdx.x;
  size_t swarmIdx = blockIdx.x;

  Swarm *swarm = &swarms[swarmIdx];
  Particle *particle = &swarm->particles[(int)tid];

  double* candidate = (double*)malloc(n * sizeof(double));
  memcpy(candidate, result, n * sizeof(double));

  // no need to copy result since it is operating on a per swarm, so its values (apart from z) will be replaced but it will not matter.
  b(result, n, swarmIdx, particle->position, candidate);

  double fitness = (*objectiveFunc)(n, candidate);

  free(candidate);

  if (fitness < particle->personalBestFitness)
  {
    particle->personalBestFitness = fitness;
    particle->personalBest = particle->position;
  }

  // START OF ATOMIC CHECKING
  __syncthreads();

  // thread 0 finds global best
  if (tid == 0) {
    for (int i = 0; i < s; i++) {
      Particle *p = &swarm->particles[i];
      if (p->personalBestFitness < swarm->globalBestFitness) {
        swarm->globalBestFitness = p->personalBestFitness;
        swarm->globalBest = p->personalBest;
      }
    }
  }

  __syncthreads();
  // END OF ATOMIC CHECKING

  // stopping condition -- Not included to measure raw machine performance
  // if (swarms[i].globalBestFitness <= threshold)
  // {
  //   return;
  // }

  // PSO Updates
  updateVelocity(particle, swarm->globalBest, iter, maxIter);
  updatePosition(particle);

}

//==============================================================

/**
 * CPSO-S Algorithm using CUDA. The blocks represent the swarm and each threads represents a particle. Since a block can only have atmost 1024 threads, each swarm is limited to have at most 1024 particles.
 * @param n dimensions
 * @param s particle size per swarm
 * @param objectiveFunction objective function where the positions of the particles will be evaluated (minima search)
 * @param maxIterations maximum iterations for when to stop the search
 * @param threshold when this value is reached (based on the objectiveFunction), search immediately stops
 *
 * @result @param result pointer, stores the best coordinate found
 */
double* CPSO_S(size_t n, int s, ObjectiveFunction objectiveFunc, int maxIterations, double threshold)
{
  int device = -1;
  cudaGetDevice (&device);

  // blocks is per dimension.
  size_t numBlocks = n;
  // max 1024 particles per swarm
  size_t numThreads = (s < 1024) ? s : 1024;

  if (s > 1024) {
    printf("Particle exceeds max CUDA thread count. Particle count is limited to 1024.");
    s = 1024;
  }

  Swarm *swarms;
  cudaMallocManaged(&swarms, n * sizeof(Swarm));

  

  for (size_t i = 0; i < n; i++) {
    cudaMallocManaged(&swarms[i].particles, s * sizeof(Particle));
  }

  double *out;
  cudaMallocManaged(&out, n * sizeof(double));

  // 1. Initialize b(j, z)
  cudaMemset(out, 0, n * sizeof(double));

  // create GPU page memory
  // cudaMemPrefetchAsync(out, n * sizeof(double), device, NULL);

  // create CPU page memory and prefetch CPU- GPU
  //cudaMemPrefetchAsync(swarms, n * sizeof(Swarm), cudaCpuDeviceId, NULL);
  // cudaMemPrefetchAsync(swarms, n * sizeof(Swarm), device, NULL); //prefetches got rid of memory thrashes (CPU - GPU)
  // for (size_t i = 0; i < n; i++) {
  //   cudaMemPrefetchAsync(swarms[i].particles, s * sizeof(Particle), cudaCpuDeviceId, NULL);
  //   cudaMemPrefetchAsync(swarms[i].particles, s * sizeof(Particle), device, NULL);
  // }

  // 2. Initialize n one-d swarms
  for (int i = 0; i < n; i++)
  {
    swarms[i].globalBest = 0;
    swarms[i].globalBestFitness = __DBL_MAX__;

    for (int j = 0; j < s; j++)
    {
      swarms[i].particles[j].position = randomInRange(POSITION_MIN, POSITION_MAX);
      swarms[i].particles[j].velocity = randomInRange(VELOCITY_MIN, VELOCITY_MAX);

      // printf("Particle %d at dimension %d, position: %f, velocity: %f\n", j, i, swarms[i].particles[j].position, swarms[i].particles[j].velocity);
      swarms[i].particles[j].personalBest = swarms[i].particles[j].position;
      out[i] = swarms[i].particles[j].personalBest;
      swarms[i].particles[j].personalBestFitness = __DBL_MAX__;
    }
  }

  // 2. Main loop
  for (int iter = 0; iter < maxIterations; iter++)
  {
    particleAction<<<numBlocks, numThreads>>>(swarms, n, s, out, iter, maxIterations, objectiveFunc);
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
      fprintf(stderr, "Kernel launch failed at iter %d: %s\n", iter, cudaGetErrorString(err));
    }

    // update out, prefetch GPU-CPU (only need out)
    // cudaMemPrefetchAsync(out, n * sizeof(double), cudaCpuDeviceId, NULL);
    cudaDeviceSynchronize(); 
    for (int i = 0; i < n; i++)
    {
      out[i] = swarms[i].globalBest;
    }
  }

  cudaDeviceSynchronize(); 

  for (int i = 0; i < n; i++)
  {
    cudaFree(swarms[i].particles);
  }

  cudaFree(swarms);
  return out;
}

int main(void)
{

  srand(time(NULL));

  // double x[] = {0, 0, 0, 0};
  // double x2[] = {1, 1, 1, 1};

  // printf("Rosenbrock at (0, 0, 0, 0) = %f\n", rosenbrock(4, x));
  // printf("Rosenbrock at (1,1,1,1) (minima) = %f\n", rosenbrock(4, x2));
  // printf("Sphere at (0, 0, 0, 0) (minima) = %f\n", sphere(4, x));
  // printf("Sphere at (1,1,1,1) = %f\n", sphere(4, x2));
  // printf("Ackley at (0, 0, 0, 0) (minima) = %f\n", ackley(4, x));
  // printf("Ackley at (1,1,1,1) = %f\n", ackley(4, x2));

  size_t n = 5;

  double* result = nullptr;

  ObjectiveFunction objectiveFunction = nullptr;
  cudaMemcpyFromSymbol(&objectiveFunction, d_sphere_ptr, sizeof(ObjectiveFunction));

  int max = 10;

  result = CPSO_S(n, 5, objectiveFunction, max, 0.25);

  cudaDeviceSynchronize();
  printf("The position (");
  for (int i = 0; i < n - 1; i++)
  {
    printf("%f,", result[i]);
  }
  printf("%f) is the best found, with a fitness value of: %f after %d iterations", result[n - 1], sphere(n, result), max);
  cudaFree(result);
  return 0;
}


Writing CUDA_cpso.cu


In [5]:
%%bash
nvcc CUDA_cpso.cu -o CUDA_cpso --expt-relaxed-constexpr -arch=sm_70 -Wno-deprecated-gpu-targets

In [6]:
%%bash
./CUDA_cpso

The position (0.045236,-0.076671,-0.114094,0.083092,0.119787) is the best found, with a fitness value of: 0.042195 after 10 iterations

In [7]:
%%bash
nvprof ./CUDA_cpso

==83173== NVPROF is profiling process 83173, command: ./CUDA_cpso


The position (0.020850,0.051729,-0.017189,0.036447,-0.030080) is the best found, with a fitness value of: 0.005639 after 10 iterations

==83173== Profiling application: ./CUDA_cpso
==83173== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   78.29%  5.5602ms        10  556.02us  256.93us  1.1660ms  particleAction(Swarm*, unsigned long, int, double*, int, int, double (*) (unsigned long, double*))
                   21.65%  1.5379ms         1  1.5379ms  1.5379ms  1.5379ms  [CUDA memset]
                    0.05%  3.7760us         1  3.7760us  3.7760us  3.7760us  [CUDA memcpy DtoH]
      API calls:   97.26%  1.40176s         1  1.40176s  1.40176s  1.40176s  cudaMemcpyFromSymbol
                    1.91%  27.541ms         7  3.9344ms  8.2150us  23.816ms  cudaMallocManaged
                    0.38%  5.4586ms        12  454.89us  3.1710us  1.0390ms  cudaDeviceSynchronize
                    0.14%  2.0720ms        10  207.20us  16.791us  1.6616ms  cudaLaunchKernel
                    0.14%  1.9691ms         1  1.9691ms  1.9691ms  1.9691ms  cudaMemset
        

In [8]:
%%writefile CUDA_cpso2.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include <assert.h>
#include <stdint.h>

#define POSITION_MIN -5
#define POSITION_MAX 5
#define VELOCITY_MAX 0.2 * POSITION_MAX
#define VELOCITY_MIN -VELOCITY_MAX // Similar to the experiment setup in the study for rosenbrack

//==============================================================
//                     STRUCT DEFINITIONS
//==============================================================
typedef struct
{
  double position;
  double velocity;
  double personalBest; // Position in dimension
  double personalBestFitness;
} Particle;

typedef struct
{
  Particle *particles;
  double globalBest;
  double globalBestFitness;
} Swarm;

typedef double (*ObjectiveFunction)(size_t, double *);


//==============================================================

//==============================================================
//                     UTILITY FUNCTIONS
//==============================================================
// CPU-function RNG_UNIFORM
static inline double RNG_UNIFORM()
{
  return (double)rand() / (double)RAND_MAX;
}
// CUDA-function RNG
__device__
static inline uint32_t mix32(uint32_t z) {
    z = (z ^ 61u) ^ (z >> 16);
    z *= 9u;
    z = z ^ (z >> 4);
    z *= 0x27d4eb2du;
    z = z ^ (z >> 15);
    return z;
}
__device__
static inline double CUDA_RNG_UNIFORM()
{
  unsigned long long t = clock64();
  uint32_t tid = blockIdx.x * blockDim.x + threadIdx.x;
  uint32_t state = mix32((uint32_t)t ^ tid);

  const uint32_t A = 1664525u;
  const uint32_t C = 1013904223u;
  uint32_t x = state;
  x = A * x + C;        // wraps mod 2^32 automatically
  state = x;

  // Use the top 24 bits for a well-distributed uniform in [0,1)
  double out = ((x >> 8) & 0x00FFFFFF) * (1.0 / 16777216.0);
  return out;
}

__device__ __host__
static inline double square(double b)
{
  return b * b;
}

__device__
static inline double clamp(double value, double min, double max)
{
  if (value < min)
  {
    return min;
  }
  if (value > max)
  {
    return max;
  }
  return value;
}

static inline double randomInRange(double min, double max)
{
  return RNG_UNIFORM() * (max - min) + min;
}

//==============================================================

//==============================================================
//              OBJECTIVE FUNCTIONS
//==============================================================
/**
 * Creates an n-dimensional search space using the Rosenbrock function
 * @param n dimensions
 * @param x coordinates
 */
__device__ __host__
double rosenbrock(size_t n, double *x)
{
  double value = 0.0;

  // f(x) = ∑(i=1 to N−1) [100(x_{i+1} − x_i^2)^2 + (1 − x_i)^2]   where   x = (x_1, …, x_N) ∈ ℝ^N
  for (size_t i = 0; i < n - 1; ++i)
  { 
    value += 100.0 * square(x[i + 1] - square(x[i])) + square(1.0 - x[i]);
  }

  return value;
}
__device__ __host__
double sphere(size_t n, double *x)
{
  double value = 0.0f;
  for (size_t i = 0; i < n; i++)
  {
    value += square(x[i]);
  }

  return value;
}

__device__ __host__
double ackley(size_t n, double *x)
{
  double a = 20.0;
  double b = 0.2;
  double c = 2.0 * M_PI;

  double sumSq = 0.0;
  double sumCos = 0.0;

  for (int i = 0; i < n; i++)
  {
    sumSq += square(x[i]);
    sumCos += cos(c * x[i]);
  }

  double term1 = -a * exp(-b * sqrt(sumSq / n));
  double term2 = -exp(sumCos / n);

  return term1 + term2 + a + exp(1);
}

// Device function-pointer symbols for objective functions
__device__ ObjectiveFunction d_rosenbrock_ptr = rosenbrock;
__device__ ObjectiveFunction d_sphere_ptr     = sphere;
__device__ ObjectiveFunction d_ackley_ptr     = ackley;

//==============================================================

//==============================================================

//==============================================================
//              ALGORITHM-DEFINED FUNCTIONS
//==============================================================
__device__
void b(double *context, size_t n, int j, double z, double *result)
{
  for (int i = 0; i < n; i++)
  {
    if (i == j)
    {
      result[i] = z;
    }
    else
    {
      result[i] = context[i];
    }
  }
}

/**
 * Velocity update function
 * v_{i,j}(t+1) = w v_{i,j}(t) + c_1 r_{1,i}(t) [ y_{i,j}(t) - x_{i,j}(t) ] + c_2 r_{2,i}(t) [ \hat{y}_j(t) - x_{i,j}(t) ]
 *
 */
__device__
void updateVelocity(Particle *particle, double globalBest, int currentIter, int maxIter)
{
  double w = 1.0 - ((double)currentIter / maxIter); // intertia weight, linear scaling
  // printf("w: %f\n", w);
  double c1 = 2; // acceleration coefficient 1
  double c2 = 2; // acceleration coefficient 2
  double r1 = CUDA_RNG_UNIFORM();
  double r2 = CUDA_RNG_UNIFORM();
  double updatedVelocity = (w * particle->velocity) + (c1 * r1 * (particle->personalBest - particle->position)) + (c2 * r2 * (globalBest - particle->position));

  particle->velocity = clamp(updatedVelocity, VELOCITY_MIN, VELOCITY_MAX);
}

/**
 * Position update function
 *
 */
__device__
void updatePosition(Particle *particle)
{
  particle->position += particle->velocity;
  particle->position = clamp(particle->position, POSITION_MIN, POSITION_MAX);
}

__global__
void particleAction(Swarm *swarms, size_t n, int s, double *result, int iter, int maxIter, ObjectiveFunction objectiveFunc)
{

  // particle index per swarm
  size_t tid = threadIdx.x;
  size_t swarmIdx = blockIdx.x;

  Swarm *swarm = &swarms[swarmIdx];
  Particle *particle = &swarm->particles[(int)tid];

  double* candidate = (double*)malloc(n * sizeof(double));
  memcpy(candidate, result, n * sizeof(double));

  // no need to copy result since it is operating on a per swarm, so its values (apart from z) will be replaced but it will not matter.
  b(result, n, swarmIdx, particle->position, candidate);

  double fitness = (*objectiveFunc)(n, candidate);

  free(candidate);

  if (fitness < particle->personalBestFitness)
  {
    particle->personalBestFitness = fitness;
    particle->personalBest = particle->position;
  }

  // START OF ATOMIC CHECKING
  __syncthreads();

  // thread 0 finds global best
  if (tid == 0) {
    for (int i = 0; i < s; i++) {
      Particle *p = &swarm->particles[i];
      if (p->personalBestFitness < swarm->globalBestFitness) {
        swarm->globalBestFitness = p->personalBestFitness;
        swarm->globalBest = p->personalBest;
      }
    }
  }

  __syncthreads();
  // END OF ATOMIC CHECKING

  // stopping condition -- Not included to measure raw machine performance
  // if (swarms[i].globalBestFitness <= threshold)
  // {
  //   return;
  // }

  // PSO Updates
  updateVelocity(particle, swarm->globalBest, iter, maxIter);
  updatePosition(particle);

}

//==============================================================

/**
 * CPSO-S Algorithm using CUDA. The blocks represent the swarm and each threads represents a particle. Since a block can only have atmost 1024 threads, each swarm is limited to have at most 1024 particles.
 * @param n dimensions
 * @param s particle size per swarm
 * @param objectiveFunction objective function where the positions of the particles will be evaluated (minima search)
 * @param maxIterations maximum iterations for when to stop the search
 * @param threshold when this value is reached (based on the objectiveFunction), search immediately stops
 *
 * @result @param result pointer, stores the best coordinate found
 */
double* CPSO_S(size_t n, int s, ObjectiveFunction objectiveFunc, int maxIterations, double threshold)
{
  int device = -1;
  cudaGetDevice (&device);

  // blocks is per dimension.
  size_t numBlocks = n;
  // max 1024 particles per swarm
  size_t numThreads = (s < 1024) ? s : 1024;

  if (s > 1024) {
    printf("Particle exceeds max CUDA thread count. Particle count is limited to 1024.");
    s = 1024;
  }

  Swarm *swarms;
  cudaMallocManaged(&swarms, n * sizeof(Swarm));

  

  for (size_t i = 0; i < n; i++) {
    cudaMallocManaged(&swarms[i].particles, s * sizeof(Particle));
  }

  double *out;
  cudaMallocManaged(&out, n * sizeof(double));

  // 1. Initialize b(j, z)
  cudaMemset(out, 0, n * sizeof(double));

  // create GPU page memory
  cudaMemPrefetchAsync(out, n * sizeof(double), device, NULL);

  // create CPU page memory and prefetch CPU- GPU
  cudaMemPrefetchAsync(swarms, n * sizeof(Swarm), cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(swarms, n * sizeof(Swarm), device, NULL); //prefetches got rid of memory thrashes (CPU - GPU)
  for (size_t i = 0; i < n; i++) {
    cudaMemPrefetchAsync(swarms[i].particles, s * sizeof(Particle), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(swarms[i].particles, s * sizeof(Particle), device, NULL);
  }

  // 2. Initialize n one-d swarms
  for (int i = 0; i < n; i++)
  {
    swarms[i].globalBest = 0;
    swarms[i].globalBestFitness = __DBL_MAX__;

    for (int j = 0; j < s; j++)
    {
      swarms[i].particles[j].position = randomInRange(POSITION_MIN, POSITION_MAX);
      swarms[i].particles[j].velocity = randomInRange(VELOCITY_MIN, VELOCITY_MAX);

      // printf("Particle %d at dimension %d, position: %f, velocity: %f\n", j, i, swarms[i].particles[j].position, swarms[i].particles[j].velocity);
      swarms[i].particles[j].personalBest = swarms[i].particles[j].position;
      out[i] = swarms[i].particles[j].personalBest;
      swarms[i].particles[j].personalBestFitness = __DBL_MAX__;
    }
  }

  // 2. Main loop
  for (int iter = 0; iter < maxIterations; iter++)
  {
    particleAction<<<numBlocks, numThreads>>>(swarms, n, s, out, iter, maxIterations, objectiveFunc);
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
      fprintf(stderr, "Kernel launch failed at iter %d: %s\n", iter, cudaGetErrorString(err));
    }

    // update out, prefetch GPU-CPU (only need out)
    cudaMemPrefetchAsync(out, n * sizeof(double), cudaCpuDeviceId, NULL);
    cudaDeviceSynchronize(); 
    for (int i = 0; i < n; i++)
    {
      out[i] = swarms[i].globalBest;
    }
  }

  cudaDeviceSynchronize(); 

  for (int i = 0; i < n; i++)
  {
    cudaFree(swarms[i].particles);
  }

  cudaFree(swarms);
  return out;
}

int main(void)
{

  srand(time(NULL));

  // double x[] = {0, 0, 0, 0};
  // double x2[] = {1, 1, 1, 1};

  // printf("Rosenbrock at (0, 0, 0, 0) = %f\n", rosenbrock(4, x));
  // printf("Rosenbrock at (1,1,1,1) (minima) = %f\n", rosenbrock(4, x2));
  // printf("Sphere at (0, 0, 0, 0) (minima) = %f\n", sphere(4, x));
  // printf("Sphere at (1,1,1,1) = %f\n", sphere(4, x2));
  // printf("Ackley at (0, 0, 0, 0) (minima) = %f\n", ackley(4, x));
  // printf("Ackley at (1,1,1,1) = %f\n", ackley(4, x2));

  size_t n = 5;

  double* result = nullptr;

  ObjectiveFunction objectiveFunction = nullptr;
  cudaMemcpyFromSymbol(&objectiveFunction, d_sphere_ptr, sizeof(ObjectiveFunction));

  int max = 10;

  result = CPSO_S(n, 5, objectiveFunction, max, 0.25);

  cudaDeviceSynchronize();
  printf("The position (");
  for (int i = 0; i < n - 1; i++)
  {
    printf("%f,", result[i]);
  }
  printf("%f) is the best found, with a fitness value of: %f after %d iterations", result[n - 1], sphere(n, result), max);
  cudaFree(result);
  return 0;
}


Writing CUDA_cpso2.cu


In [9]:
%%bash
nvcc CUDA_cpso2.cu -o CUDA_cpso2 --expt-relaxed-constexpr -arch=sm_70 -Wno-deprecated-gpu-targets

In [10]:
%%bash
nvprof ./CUDA_cpso2

==83227== NVPROF is profiling process 83227, command: ./CUDA_cpso2


The position (0.016701,-0.010746,0.020765,0.001714,0.092299) is the best found, with a fitness value of: 0.009348 after 10 iterations

==83227== Profiling application: ./CUDA_cpso2
==83227== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   54.89%  4.0484ms         1  4.0484ms  4.0484ms  4.0484ms  [CUDA memset]
                   45.07%  3.3242ms        10  332.42us  169.18us  793.82us  particleAction(Swarm*, unsigned long, int, double*, int, int, double (*) (unsigned long, double*))
                    0.04%  3.2000us         1  3.2000us  3.2000us  3.2000us  [CUDA memcpy DtoH]
      API calls:   95.27%  927.45ms         1  927.45ms  927.45ms  927.45ms  cudaMemcpyFromSymbol
                    2.67%  25.952ms         7  3.7075ms  6.8540us  22.235ms  cudaMallocManaged
                    0.52%  5.0880ms        12  424.00us  6.0340us  1.0457ms  cudaDeviceSynchronize
                    0.51%  4.9954ms        23  217.19us  44.439us  1.5697ms  cudaMemPrefetchAsync
                    0.46%  4.4658ms         1  4.4658ms  4.4658ms  4.4658ms  cudaMemset
   

In [11]:
%%writefile CUDA_cpso3.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include <assert.h>
#include <stdint.h>

#define POSITION_MIN -5
#define POSITION_MAX 5
#define VELOCITY_MAX 0.2 * POSITION_MAX
#define VELOCITY_MIN -VELOCITY_MAX // Similar to the experiment setup in the study for rosenbrack

//==============================================================
//                     STRUCT DEFINITIONS
//==============================================================
typedef struct
{
  double position;
  double velocity;
  double personalBest; // Position in dimension
  double personalBestFitness;
} Particle;

typedef struct
{
  Particle *particles;
  double globalBest;
  double globalBestFitness;
} Swarm;

typedef double (*ObjectiveFunction)(size_t, double *);

//==============================================================

//==============================================================
//                     UTILITY FUNCTIONS
//==============================================================
// CPU-function RNG_UNIFORM
static inline double RNG_UNIFORM()
{
  return (double)rand() / (double)RAND_MAX;
}
// CUDA-function RNG
__device__
static inline uint32_t mix32(uint32_t z) {
    z = (z ^ 61u) ^ (z >> 16);
    z *= 9u;
    z = z ^ (z >> 4);
    z *= 0x27d4eb2du;
    z = z ^ (z >> 15);
    return z;
}
__device__
static inline double CUDA_RNG_UNIFORM()
{
  unsigned long long t = clock64();
  uint32_t tid = blockIdx.x * blockDim.x + threadIdx.x;
  uint32_t state = mix32((uint32_t)t ^ tid);

  const uint32_t A = 1664525u;
  const uint32_t C = 1013904223u;
  uint32_t x = state;
  x = A * x + C;        // wraps mod 2^32 automatically
  state = x;

  // Use the top 24 bits for a well-distributed uniform in [0,1)
  double out = ((x >> 8) & 0x00FFFFFF) * (1.0 / 16777216.0);
  return out;
}

__device__ __host__
static inline double square(double b)
{
  return b * b;
}

__device__
static inline double clamp(double value, double min, double max)
{
  if (value < min)
  {
    return min;
  }
  if (value > max)
  {
    return max;
  }
  return value;
}

static inline double randomInRange(double min, double max)
{
  return RNG_UNIFORM() * (max - min) + min;
}

//==============================================================

//==============================================================
//              OBJECTIVE FUNCTIONS
//==============================================================
/**
 * Creates an n-dimensional search space using the Rosenbrock function
 * @param n dimensions
 * @param x coordinates
 */
__device__ __host__
double rosenbrock(size_t n, double *x)
{
  double value = 0.0;

  // f(x) = ∑(i=1 to N−1) [100(x_{i+1} − x_i^2)^2 + (1 − x_i)^2]   where   x = (x_1, …, x_N) ∈ ℝ^N
  for (size_t i = 0; i < n - 1; ++i)
  { 
    value += 100.0 * square(x[i + 1] - square(x[i])) + square(1.0 - x[i]);
  }

  return value;
}
__device__ __host__
double sphere(size_t n, double *x)
{
  double value = 0.0f;
  for (size_t i = 0; i < n; i++)
  {
    value += square(x[i]);
  }

  return value;
}

__device__ __host__
double ackley(size_t n, double *x)
{
  double a = 20.0;
  double b = 0.2;
  double c = 2.0 * M_PI;

  double sumSq = 0.0;
  double sumCos = 0.0;

  for (int i = 0; i < n; i++)
  {
    sumSq += square(x[i]);
    sumCos += cos(c * x[i]);
  }

  double term1 = -a * exp(-b * sqrt(sumSq / n));
  double term2 = -exp(sumCos / n);

  return term1 + term2 + a + exp(1);
}

// Device function-pointer symbols for objective functions
__device__ ObjectiveFunction d_rosenbrock_ptr = rosenbrock;
__device__ ObjectiveFunction d_sphere_ptr     = sphere;
__device__ ObjectiveFunction d_ackley_ptr     = ackley;

//==============================================================

//==============================================================

//==============================================================
//              ALGORITHM-DEFINED FUNCTIONS
//==============================================================
__device__
void b(double *context, size_t n, int j, double z, double *result)
{
  for (int i = 0; i < n; i++)
  {
    if (i == j)
    {
      result[i] = z;
    }
    else
    {
      result[i] = context[i];
    }
  }
}

/**
 * Velocity update function
 * v_{i,j}(t+1) = w v_{i,j}(t) + c_1 r_{1,i}(t) [ y_{i,j}(t) - x_{i,j}(t) ] + c_2 r_{2,i}(t) [ \hat{y}_j(t) - x_{i,j}(t) ]
 *
 */
__device__
void updateVelocity(Particle *particle, double globalBest, int currentIter, int maxIter)
{
  double w = 1.0 - ((double)currentIter / maxIter); // intertia weight, linear scaling
  // printf("w: %f\n", w);
  double c1 = 2; // acceleration coefficient 1
  double c2 = 2; // acceleration coefficient 2
  double r1 = CUDA_RNG_UNIFORM();
  double r2 = CUDA_RNG_UNIFORM();
  double updatedVelocity = (w * particle->velocity) + (c1 * r1 * (particle->personalBest - particle->position)) + (c2 * r2 * (globalBest - particle->position));

  particle->velocity = clamp(updatedVelocity, VELOCITY_MIN, VELOCITY_MAX);
}

/**
 * Position update function
 *
 */
__device__
void updatePosition(Particle *particle)
{
  particle->position += particle->velocity;
  particle->position = clamp(particle->position, POSITION_MIN, POSITION_MAX);
}

__global__
void particleAction(Swarm *swarms, size_t n, int s, double *result, int iter, int maxIter, ObjectiveFunction objectiveFunc)
{

  // particle index per swarm
  size_t tid = threadIdx.x;
  size_t swarmIdx = blockIdx.x;

  Swarm *swarm = &swarms[swarmIdx];
  Particle *particle = &swarm->particles[(int)tid];

  double* candidate = (double*)malloc(n * sizeof(double));
  memcpy(candidate, result, n * sizeof(double));

  // no need to copy result since it is operating on a per swarm, so its values (apart from z) will be replaced but it will not matter.
  b(result, n, swarmIdx, particle->position, candidate);

  double fitness = (*objectiveFunc)(n, candidate);

  free(candidate);

  if (fitness < particle->personalBestFitness)
  {
    particle->personalBestFitness = fitness;
    particle->personalBest = particle->position;
  }

  // START OF ATOMIC CHECKING
  __syncthreads();

  // thread 0 finds global best
  if (tid == 0) {
    for (int i = 0; i < s; i++) {
      Particle *p = &swarm->particles[i];
      if (p->personalBestFitness < swarm->globalBestFitness) {
        swarm->globalBestFitness = p->personalBestFitness;
        swarm->globalBest = p->personalBest;
      }
    }
  }

  __syncthreads();
  // END OF ATOMIC CHECKING

  // stopping condition -- Not included to measure raw machine performance
  // if (swarms[i].globalBestFitness <= threshold)
  // {
  //   return;
  // }

  // PSO Updates
  updateVelocity(particle, swarm->globalBest, iter, maxIter);
  updatePosition(particle);

}

//==============================================================

/**
 * CPSO-S Algorithm using CUDA. The blocks represent the swarm and each threads represents a particle. Since a block can only have atmost 1024 threads, each swarm is limited to have at most 1024 particles.
 * @param n dimensions
 * @param s particle size per swarm
 * @param objectiveFunction objective function where the positions of the particles will be evaluated (minima search)
 * @param maxIterations maximum iterations for when to stop the search
 * @param threshold when this value is reached (based on the objectiveFunction), search immediately stops
 *
 * @result @param result pointer, stores the best coordinate found
 */
double* CPSO_S(size_t n, int s, ObjectiveFunction objectiveFunc, int maxIterations, double threshold)
{
  int device = -1;
  cudaGetDevice (&device);

  // blocks is per dimension.
  size_t numBlocks = n;
  // max 1024 particles per swarm
  size_t numThreads = (s < 1024) ? s : 1024;

  if (s > 1024) {
    printf("Particle exceeds max CUDA thread count. Particle count is limited to 1024.");
    s = 1024;
  }

  Swarm *swarms;
  cudaMallocManaged(&swarms, n * sizeof(Swarm));

  cudaMemAdvise(swarms, n * sizeof(Swarm), cudaMemAdviseSetPreferredLocation, device);
  cudaMemAdvise(swarms, n * sizeof(Swarm), cudaMemAdviseSetAccessedBy, device);


  for (size_t i = 0; i < n; i++) {
    cudaMallocManaged(&swarms[i].particles, s * sizeof(Particle));
    cudaMemAdvise(swarms[i].particles, s * sizeof(Particle), cudaMemAdviseSetPreferredLocation, device);
    cudaMemAdvise(swarms[i].particles, s * sizeof(Particle), cudaMemAdviseSetAccessedBy, device);
  }

  double *out;
  cudaMallocManaged(&out, n * sizeof(double));

  // 1. Initialize b(j, z)
  cudaMemset(out, 0, n * sizeof(double));


  // create CPU page memory and prefetch CPU- GPU
  cudaMemPrefetchAsync(swarms, n * sizeof(Swarm), cudaCpuDeviceId, NULL);
  for (size_t i = 0; i < n; i++) {
    cudaMemPrefetchAsync(swarms[i].particles, s * sizeof(Particle), cudaCpuDeviceId, NULL);
  }

  //  prefetch CPU- GPU
  cudaMemPrefetchAsync(swarms, n * sizeof(Swarm), device, NULL); //prefetches got rid of memory thrashes (CPU - GPU)
  for (size_t i = 0; i < n; i++) {
    cudaMemPrefetchAsync(swarms[i].particles, s * sizeof(Particle), device, NULL);
  }
  cudaMemPrefetchAsync(out, n * sizeof(double), device, NULL);

  cudaDeviceSynchronize();
  
  // 2. Initialize n one-d swarms
  for (int i = 0; i < n; i++)
  {
    swarms[i].globalBest = 0;
    swarms[i].globalBestFitness = __DBL_MAX__;

    for (int j = 0; j < s; j++)
    {
      swarms[i].particles[j].position = randomInRange(POSITION_MIN, POSITION_MAX);
      swarms[i].particles[j].velocity = randomInRange(VELOCITY_MIN, VELOCITY_MAX);

      // printf("Particle %d at dimension %d, position: %f, velocity: %f\n", j, i, swarms[i].particles[j].position, swarms[i].particles[j].velocity);
      swarms[i].particles[j].personalBest = swarms[i].particles[j].position;
      out[i] = swarms[i].particles[j].personalBest;
      swarms[i].particles[j].personalBestFitness = __DBL_MAX__;
    }
  }

  // 2. Main loop
  for (int iter = 0; iter < maxIterations; iter++)
  {
    particleAction<<<numBlocks, numThreads>>>(swarms, n, s, out, iter, maxIterations, objectiveFunc);
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
      fprintf(stderr, "Kernel launch failed at iter %d: %s\n", iter, cudaGetErrorString(err));
    }

    // update out, prefetch GPU-CPU (only need out)
    cudaMemPrefetchAsync(out, n * sizeof(double), cudaCpuDeviceId, NULL);
    cudaDeviceSynchronize(); 
    for (int i = 0; i < n; i++)
    {
      out[i] = swarms[i].globalBest;
    }
    cudaMemPrefetchAsync(out, n * sizeof(double), device, NULL);

  }

  cudaDeviceSynchronize(); 

  for (int i = 0; i < n; i++)
  {
    cudaFree(swarms[i].particles);
  }

  cudaFree(swarms);
  return out;
}

int main(void)
{

  srand(time(NULL));

  // double x[] = {0, 0, 0, 0};
  // double x2[] = {1, 1, 1, 1};

  // printf("Rosenbrock at (0, 0, 0, 0) = %f\n", rosenbrock(4, x));
  // printf("Rosenbrock at (1,1,1,1) (minima) = %f\n", rosenbrock(4, x2));
  // printf("Sphere at (0, 0, 0, 0) (minima) = %f\n", sphere(4, x));
  // printf("Sphere at (1,1,1,1) = %f\n", sphere(4, x2));
  // printf("Ackley at (0, 0, 0, 0) (minima) = %f\n", ackley(4, x));
  // printf("Ackley at (1,1,1,1) = %f\n", ackley(4, x2));

  size_t n = 5;

  double* result = nullptr;

  ObjectiveFunction objectiveFunction = nullptr;
  cudaMemcpyFromSymbol(&objectiveFunction, d_sphere_ptr, sizeof(ObjectiveFunction));

  int max = 10;

  result = CPSO_S(n, 5, objectiveFunction, max, 0.25);

  cudaDeviceSynchronize();
  printf("The position (");
  for (int i = 0; i < n - 1; i++)
  {
    printf("%f,", result[i]);
  }
  printf("%f) is the best found, with a fitness value of: %f after %d iterations", result[n - 1], sphere(n, result), max);
  cudaFree(result);
  return 0;
}


Writing CUDA_cpso3.cu


In [12]:
%%bash
nvcc CUDA_cpso3.cu -o CUDA_cpso3 --expt-relaxed-constexpr -arch=sm_70 -Wno-deprecated-gpu-targets

In [13]:
%%bash
nvprof ./CUDA_cpso3

==83284== NVPROF is profiling process 83284, command: ./CUDA_cpso3


The position (0.053934,-0.095681,0.037735,0.016609,-0.021366) is the best found, with a fitness value of: 0.014220 after 10 iterations

==83284== Profiling application: ./CUDA_cpso3
==83284== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   51.26%  2.0166ms        10  201.66us  51.903us  1.5247ms  particleAction(Swarm*, unsigned long, int, double*, int, int, double (*) (unsigned long, double*))
                   48.64%  1.9138ms         1  1.9138ms  1.9138ms  1.9138ms  [CUDA memset]
                    0.10%  3.8080us         1  3.8080us  3.8080us  3.8080us  [CUDA memcpy DtoH]
      API calls:   96.46%  1.42034s         1  1.42034s  1.42034s  1.42034s  cudaMemcpyFromSymbol
                    1.73%  25.411ms         7  3.6301ms  10.073us  23.748ms  cudaMallocManaged
                    0.70%  10.254ms        33  310.74us  22.860us  1.8353ms  cudaMemPrefetchAsync
                    0.46%  6.7064ms        13  515.88us  15.086us  1.1185ms  cudaDeviceSynchronize
                    0.27%  4.0381ms        10  403.81us  108.74us  2.2172ms  cudaLaunchKern